In [84]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



import csv
import os
from dotenv import load_dotenv

In [85]:
def update_csv_file(driver, installation_count, apk_link, file_name):
    #pkg_name perchè mi assicuro aggiunga nella riga esatta + "/download/apk"
    #installation_count è la variabile da aggiungere
    
    #Percorso del file CSV
    csv_file_path = "/home/pepe/visualStudioCodeProjects/tirocinio/CSV_category/" + file_name


    # Leggi il file CSV
    df = pd.read_csv(csv_file_path)

    # Verifica se la colonna "installations" esiste, altrimenti creala
    if "installations" not in df.columns:
        df["installations"] = None

    # Trova la riga dove apk_link corrisponde al valore cercato e aggiorna la colonna "installations"
    df.loc[df["apk_link"] == apk_link, "installations"] = installation_count

    # Salva le modifiche nel file CSV
    df.to_csv(csv_file_path, index=False)

    print(f"Valore di {installation_count} aggiornato per pkg_name {apk_link} nella colonna installations.")
    driver.quit()


In [86]:
#Funzione per cercare l'app su APKCombo e ottenere il numero di download
def get_download_count(driver, pkg_name, file_name):
    try:
        # Accedi a apkcombo.com
        pkg_name_no_add = pkg_name.replace("/download/apk", "")
        print(pkg_name_no_add)
        driver.get(pkg_name_no_add)

         #wait until non trova ID
        WebDriverWait(driver,5).until(
            EC.presence_of_element_located((By.XPATH, '//button[@aria-label="Consent"]'))
        ) #aspetta 5 secondi prima di cercare searchInput
        l = driver.find_element(By.XPATH, '//button[@aria-label="Consent"]')
        c = l.click()

        installation_count = driver.find_element(By.XPATH, '//div[contains(@class, "item") and .//div[text()="Installs"]]//div[@class="value"]').text.replace("+", "")
        print(installation_count + ": number of installations")
        update_csv_file(driver, installation_count, pkg_name, file_name)

    except Exception as e:
        print(f"Errore durante la ricerca di {pkg_name}: {e}")
        return None

In [87]:

def lookForApp(appName, file_name):
    # Avvio del browser
    load_dotenv()

    chromeDriverPath = os.getenv('webdriver_chrome_driver')

    #Connecting to play store
    service = Service(executable_path=chromeDriverPath)
    #service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-third-party-cookies")

    get_download_count(driver, appName, file_name)
    #driver.get("https://google.com")

In [88]:
# Funzione per processare ogni file CSV e ottenere le stringhe sotto la colonna "pkg_name"
def process_csv_files(folder_path):
    # Itera su tutti i file CSV nella cartella
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):  # Controlla che sia un file CSV
            print("Ora nel csv:" + file_name)
            file_path = os.path.join(folder_path, file_name)
            
            # Leggi il file CSV
            df = pd.read_csv(file_path)
            
            # Controlla se la colonna "pkg_name" esiste
            if "apk_link" in df.columns:
                print(f"Processing file: {file_name}")
                
                # Itera su ogni valore della colonna "pkg_name"
                for index, pkg_name in enumerate(df["apk_link"]):
                    lookForApp(pkg_name, file_name)
            else:
                print(f"Warning: 'pkg_name' column not found in {file_name}")

In [89]:
# Percorso dei file CSV
csv_folder_path = "/home/pepe/visualStudioCodeProjects/tirocinio/CSV_category"


# Esegui il processo sui file CSV nella cartella specificata
process_csv_files(csv_folder_path)



# Chiudi il browser
#driver.quit()

Ora nel csv:personalization_apk_popular.csv
Processing file: personalization_apk_popular.csv
https://apkcombo.com/icedrive-secure-cloud-storage/com.icedrive.app
100,000: number of installations
Valore di 100,000 aggiornato per pkg_name https://apkcombo.com/icedrive-secure-cloud-storage/com.icedrive.app/download/apk nella colonna installations.


/tmp/ipykernel_53298/1193380557.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '100,000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df["apk_link"] == apk_link, "installations"] = installation_count


https://apkcombo.com/terabox/com.dubox.drive
100,000,000: number of installations
Valore di 100,000,000 aggiornato per pkg_name https://apkcombo.com/terabox/com.dubox.drive/download/apk nella colonna installations.
https://apkcombo.com/cloud-storage/hr.telekomcloud.storage
10,000: number of installations
Valore di 10,000 aggiornato per pkg_name https://apkcombo.com/cloud-storage/hr.telekomcloud.storage/download/apk nella colonna installations.
https://apkcombo.com/degoo-20-gb-cloud-storage/com.degoo.android
10,000,000: number of installations
Valore di 10,000,000 aggiornato per pkg_name https://apkcombo.com/degoo-20-gb-cloud-storage/com.degoo.android/download/apk nella colonna installations.
https://apkcombo.com/yandex-disk-file-cloud-storage/ru.yandex.disk


KeyboardInterrupt: 